***
## **<center>Spark x Pandas</center>**


**<center>Processamento de Dados Massivos</center>**
***

<br>
Este notebook tem o objetivo de apresentar um caso onde, devido ao tamanho do conjunto de dados, utilizar o Spark se torna mandatório com relação ao Pandas.

Trata-se de uma base pública de reclamações de usuários de empresas de telecomunicações junto à Anatel. Se tentarmos criar um dataframe Pandas, a memória pode não aguentar... O que aconteceria com o Spark??

In [1]:
import os
os.environ["JAVA_HOME"] = "C:\\jdk-22.0.2"
os.environ["SPARK_HOME"] = "C:\\spark-3.5.2-bin-hadoop3\\spark-3.5.2-bin-hadoop3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_PYTHON"] = "python"

### Primeira Parte) Tentando com o Pandas

In [2]:
link_reclamacoes = 'https://www.anatel.gov.br/dadosabertos/paineis_de_dados/consumidor/consumidor_reclamacoes.zip'

In [3]:
import pandas as pd

df = pd.read_csv('C:\\01-FaculdadeSemestreAtual\\Processamento_de_Dados_Massivos\\reclamacoes_contexto.csv', sep = ';')

C:\Users\guiro\AppData\Local\Temp\ipykernel_2568\1200418084.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:\\01-FaculdadeSemestreAtual\\Processamento_de_Dados_Massivos\\reclamacoes_contexto.csv', sep = ';')


In [4]:
df.head()

,DataExtracao,SOLICITAÇÕES,Ano,Mês,AnoMês,UF,Cidade,CO_MUNICIPIO,CanalEntrada,Condição,TipoAtendimento,Serviço,Marca,Assunto,Problema,Linha
0,2018-04-23,1,2015,1,2015-01,DF,Brasília,5300108,SIC,Reencaminhada,Pedido de Informação,Outros,Outros,NÃO APLICÁVEL,SIC,1
1,2018-04-23,1,2015,1,2015-01,AC,Rio Branco,1200401,Fale Conosco,Nova,Reclamação,STFC,OI,Bloqueio ou Suspensão e Desbloqueio,Bloqueio ou suspensão indevido,2
2,2018-04-23,1,2015,1,2015-01,AL,Maceió,2704302,Fale Conosco,Nova,Reclamação,SMP_Pós,VIVO,Qualidade e Funcionamento do Serviço ou Equipa...,Lentidão ou velocidade reduzida de conexão,3
3,2018-04-23,1,2015,1,2015-01,AM,Manaus,1302603,Fale Conosco,Reencaminhada,Reclamação,SMP_Pré,VIVO,Crédito Pré-pago,Crédito pré-pago cobrado de maneira indevida,4
4,2018-04-23,1,2015,1,2015-01,AM,Manaus,1302603,Fale Conosco,Nova,Reclamação,STFC,OI,Qualidade e Funcionamento do Serviço ou Equipa...,Interrupção do serviço,5


### Segunda Parte) Agora com o Spark...

In [5]:
import pyspark
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName("Introducao Spark").getOrCreate()

In [8]:
df_spark = spark.read.option("header","true")\
          .option("encoding", "utf-8")\
          .option("sep",";")\
          .option('inferSchema', 'true')\
          .csv('C:\\01-FaculdadeSemestreAtual\\Processamento_de_Dados_Massivos\\reclamacoes_contexto.csv')


In [9]:
df_spark.printSchema()

root
 |-- DataExtracao: date (nullable = true)
 |-- SOLICITAÇÕES: integer (nullable = true)
 |-- Ano: integer (nullable = true)
 |-- Mês: integer (nullable = true)
 |-- AnoMês: timestamp (nullable = true)
 |-- UF: string (nullable = true)
 |-- Cidade: string (nullable = true)
 |-- CO_MUNICIPIO: string (nullable = true)
 |-- CanalEntrada: string (nullable = true)
 |-- Condição: string (nullable = true)
 |-- TipoAtendimento: string (nullable = true)
 |-- Serviço: string (nullable = true)
 |-- Marca: string (nullable = true)
 |-- Assunto: string (nullable = true)
 |-- Problema: string (nullable = true)
 |-- Linha: integer (nullable = true)



In [12]:
# Quantos registros no total?
df_spark.count()

22513536

In [10]:
# Uso do comando "limit()"
df_spark.limit(5).show()

+------------+------------+----+---+-------------------+---+----------+------------+------------+-------------+--------------------+-------+------+--------------------+--------------------+-----+
|DataExtracao|SOLICITAÇÕES| Ano|Mês|             AnoMês| UF|    Cidade|CO_MUNICIPIO|CanalEntrada|     Condição|     TipoAtendimento|Serviço| Marca|             Assunto|            Problema|Linha|
+------------+------------+----+---+-------------------+---+----------+------------+------------+-------------+--------------------+-------+------+--------------------+--------------------+-----+
|  2018-04-23|           1|2015|  1|2015-01-01 00:00:00| DF|  Brasília|     5300108|         SIC|Reencaminhada|Pedido de Informação| Outros|Outros|       NÃO APLICÁVEL|                 SIC|    1|
|  2018-04-23|           1|2015|  1|2015-01-01 00:00:00| AC|Rio Branco|     1200401|Fale Conosco|         Nova|          Reclamação|   STFC|    OI|Bloqueio ou Suspe...|Bloqueio ou suspe...|    2|
|  2018-04-23|      

In [17]:
# Quantas reclamações por UF?
df_spark.limit(1000).groupBy("UF").agg({"SOLICITAÇÕES": "sum"}).orderBy("sum(SOLICITAÇÕES)").show()


+---+-----------------+
| UF|sum(SOLICITAÇÕES)|
+---+-----------------+
| RN|                1|
| PI|                1|
| MT|                2|
| RR|                2|
| TO|                2|
| MA|                4|
| PA|                4|
| ES|                5|
| AP|                5|
| AC|                6|
| MS|                7|
| SE|                7|
| RO|               10|
| PB|               12|
| SC|               16|
| PE|               16|
| DF|               17|
| AL|               20|
| RS|               23|
| PR|               24|
+---+-----------------+
only showing top 20 rows



In [14]:
# Quantas reclamações da Vivo?
df_spark.filter((df_spark.Marca == 'VIVO')).show(truncate=False)

+------------+------------+----+---+-------------------+---+--------------------+------------+------------+-------------+---------------+-------+-----+-----------------------------------------------------+-------------------------------------------------------------+-----+
|DataExtracao|SOLICITAÇÕES|Ano |Mês|AnoMês             |UF |Cidade              |CO_MUNICIPIO|CanalEntrada|Condição     |TipoAtendimento|Serviço|Marca|Assunto                                              |Problema                                                     |Linha|
+------------+------------+----+---+-------------------+---+--------------------+------------+------------+-------------+---------------+-------+-----+-----------------------------------------------------+-------------------------------------------------------------+-----+
|2018-04-23  |1           |2015|1  |2015-01-01 00:00:00|AL |Maceió              |2704302     |Fale Conosco|Nova         |Reclamação     |SMP_Pós|VIVO |Qualidade e Funcionamento d

In [21]:
solicitacoes_anuais = df_spark.groupby('Ano').agg({'SOLICITAÇÕES': 'sum'}).toPandas()
solicitacoes_anuais

,Ano,sum(SOLICITAÇÕES)
0,2015,4132965
1,2017,3447450
2,2016,3962021
3,2018,2982349
4,2019,3018934
5,2020,3007538
6,2021,2305937
7,2022,1785635
8,2023,1360601
9,2024,775860
